In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras import layers, activations
from os import path
from PIL import Image
import pickle
from concurrent import futures
from matplotlib import pyplot as plt

trainingDataDir = 'o:/temp/pixiv/training/'
targetSize = (224, 224)
targetShape = (224, 224, 3)
seed = 1

In [ ]:
data = pd.read_csv(path.join(trainingDataDir, 'score.csv'),index_col='img')
data.head()

In [ ]:
# create training and validation set
data1 = data.sample(frac=0.8, random_state=seed)
data2 = data.drop(data1.index)

In [ ]:
def process_img(imgName):
    img = Image.open(path.join(trainingDataDir, imgName))
    xx = max(img.size)
    newImg = Image.new('RGB', (xx, xx))
    newImg.paste(img, (int((xx - img.size[0]) / 2), int(
        (xx - img.size[1]) / 2)))
    newImg = newImg.resize(targetSize, Image.BICUBIC)
    return np.array(newImg)


def create_dateset(data):
    # dx,dy = ([],[])
    with futures.ThreadPoolExecutor(max_workers=12) as executor:
        trainX = np.array(list(executor.map(process_img, data.index)))
        trainY = []
        for i in data.index:
            score = int(data.loc[i, 'score'])
            scoreOneHot = np.zeros(3)
            scoreOneHot[score - 1] = 1
            trainY.append(scoreOneHot)
        trainY = np.array(trainY)
        return trainX, trainY


def create_datasets(data):
    data1 = data.sample(frac=0.8, random_state=seed)
    data2 = data.drop(data1.index)
    (trainX, trainY) = create_dateset(data1)
    (testX, testY) = create_dateset(data2)
    return (trainX, trainY), (testX, testY)


def load_data(dataPath):
    if path.exists(dataPath):
        with open(dataPath, 'rb') as f:
            return pickle.load(f)
    else:
        with open('ds.pickle', 'wb') as f:
            (trainX, trainY), (testX, testY) = create_datasets(data)
            pickle.dump(((trainX, trainY), (testX, testY)), f)
        return (trainX, trainY), (testX, testY)

In [ ]:
(trainX, trainY), (testX, testY) = load_data('ds.pickle')

In [ ]:
mobNet = keras.applications.mobilenet_v2.MobileNetV2(include_top=False,
                                                     weights='imagenet',
                                                     input_shape=targetShape)
for layer in mobNet.layers:
    layer.trainable = False


In [ ]:
inputLayer = layers.Input(shape=targetShape)
# preprocess
x = layers.RandomFlip()(inputLayer)
x = layers.RandomZoom(height_factor=0.2, width_factor=0.2)(x)
x = layers.RandomRotation(0.2)(x)
x = layers.Rescaling(scale=1. / 127.5, offset=-1)(x)
x = mobNet(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2048)(x)
x = layers.ReLU()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2048)(x)
x = layers.ReLU()(x)
outputLayer = layers.Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputLayer, outputs=outputLayer)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
cbs = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                  patience=20,
                                  verbose=1,
                                  restore_best_weights=True)
]
model.fit(trainX, trainY, batch_size=64, epochs=1000, validation_split=0.2,callbacks=cbs)


In [ ]:
# model = keras.models.load_model('model-dp2-06280.h5')

In [ ]:
import random
i = random.randint(0, len(testX))
pred = model.predict(testX[i:i + 1])
print(pred)
print('pred:', np.argmax(pred[0]) + 1)
print('y:', np.argmax(testY[i:i + 1]) + 1)
plt.imshow(testX[i])


In [ ]:
model.evaluate(testX, testY)

In [ ]:
model.save('model-dp2.h5')